In [7]:
# Load input into numpy array.

import numpy

grid = [
            # Create a list of each number, based on each line in our input.
            list(x.strip()) for x in open('input.txt') # Do this for every line in our open input.
]  # Make it a list of lists.

tree_array = numpy.array(grid)  # Turn this into an ndarray.

In [8]:

alignment_array_store = []  # Store our newly generated arrays.

# For each alignment of our grid (4, N,S,E and W)...
for rotations, alignment in enumerate(range(4)):

    # We make a destructive copy of our tree_array.
    alignment_array = tree_array.copy()
    
    # We rotate the alignment by the alignment iteration counter-clockwise.
    alignment_array = numpy.rot90(alignment_array, rotations, axes=(0,1))
    
    for x, row in enumerate(alignment_array):  # For each row in the copied array:
        
        previous_tree_height = -1  # Reset previous tree height at start of every row.

        for y, current_tree in enumerate(row):
            
            # Take the current tree's height.
            current_tree_height = int(current_tree)

            # Is the number greater than the previous tree's number?
            visible = current_tree_height > previous_tree_height
            
            # Assign to our array.
            alignment_array[x][y] = visible
            
            # Assign current tree to previous tree if it's taller.
            if visible:
                previous_tree_height = current_tree_height
    
    # Rotate our alignment_array back to the original position.
    
    alignment_array = numpy.rot90(alignment_array, rotations, axes=(1,0))
    
    # Append it to our array store.
    alignment_array_store.append(alignment_array)

In [9]:
# Flatten our arrays in our alignment store to one array.

# NOTE: In review, we wouldn't have to run a replace if our data types were correct at the start.

import pandas

visibility_map = pandas.DataFrame(alignment_array_store[0]).replace("T", True).replace("F", False)

for array in alignment_array_store[1:]:
    array_dataframe = pandas.DataFrame(array).replace("T", True).replace("F", False)
    visibility_map = visibility_map + array_dataframe
    
visibility_map

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,False,False,False,True,False,False,True,False,False,...,False,True,False,False,True,False,True,False,False,True
2,True,False,False,False,False,False,False,True,True,False,...,False,False,True,False,True,False,False,False,False,True
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,True,True,False,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,True,False,False,False,True,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,True
95,True,False,True,True,True,True,False,False,True,False,...,True,True,True,False,False,False,False,False,True,True
96,True,True,False,False,True,True,True,True,False,False,...,False,False,False,False,True,False,False,False,True,True
97,True,False,True,False,False,True,True,False,True,False,...,False,True,True,False,False,False,False,True,False,True


In [10]:
# Count the 'trues'. Part 1 answer.
visibility_map.sum().sum()

1763

In [11]:
# We make a destructive copy of our tree_array. We can't rotate, or our indexes get messed up.
destructive_array = numpy.asarray(tree_array.copy(), dtype=int)
editable_array = numpy.zeros_like(destructive_array)

for row_number, row in enumerate(destructive_array):  # For each row in the copied array:
    for column_number, current_tree in enumerate(row):
        # How many trees will it see, before getting to a tree the same size as itself or larger? Count the tree that blocks it.

        right_score_counter = 1 # Reset score counter to 1.
        left_score_counter = 1 # Reset score counter to 1.
        up_score_counter = 1 # Reset score counter to 1.
        down_score_counter = 1 # Reset score counter to 1.

        ### Left to Right ###
        
        # Get a list of trees that our tree can potentially see, starting one after it's position on the current row, to the end of the row.
        for score, viewable_tree in enumerate(row[column_number+1:], start=1):
            right_score_counter = score
                
            # If the current tree is same size or shorter than the viewable tree, we break.
            if int(current_tree) <= int(viewable_tree):
                break
        
        ### Right to Left ###
        
        # Get a list of trees that our tree can potentially see, starting one after it's position on the current row, to the end of the row.
        for score, viewable_tree in enumerate(numpy.flip(row)[len(row)-column_number:], start=1):
            left_score_counter = score
                
            # If the current tree is same size or shorter than the viewable tree, we break.
            if int(current_tree) <= int(viewable_tree):
                break
                
        ### Up to Down ###
        
        # Get a list of trees that our tree can potentially see, starting one after it's position on the current row, to the end of the row.
        for score, viewable_tree in enumerate(destructive_array[row_number+1:, column_number], start=1):
            down_score_counter = score
                
            # If the current tree is same size or shorter than the viewable tree, we break.
            if int(current_tree) <= int(viewable_tree):
                break
            
        ### Down to Up ###
        
        # Get a list of trees that our tree can potentially see, starting one after it's position on the current row, to the end of the row.
        entire_column = list(destructive_array[:, column_number])
        
        # Reverse our column.
        entire_column.reverse()
        
        segment_of_column = entire_column[99-row_number:]
        
        for score, viewable_tree in enumerate(segment_of_column, start=1):
            up_score_counter = score
                
            # print("=====")
            # print(f"The row of the tree is: {row_number} and the column is {column_number}")
            # print((row_number, column_number))
            
            # print(f"The tree's value is: {current_tree}")
            
            # print("The entire column is:")
            # print(entire_column)
            
            # print("The segment it has picked is:")
            # print(segment_of_column)
                
            # If the current tree is same size or shorter than the viewable tree, we break.
            if int(current_tree) <= int(viewable_tree):
                break
        
        scores_total = up_score_counter * down_score_counter * left_score_counter * right_score_counter
        
        # print("====")
        # print((row_number, column_number))
        # print(up_score_counter)
        # print(down_score_counter)
        # print(left_score_counter)
        # print(right_score_counter)
        # print(f"Total: {scores_total}")

        # Assign score to our array.
        editable_array[column_number][row_number] = scores_total

editable_array

array([[ 1,  2, 42, ...,  1,  4,  2],
       [ 3,  2,  1, ..., 12,  1,  1],
       [ 1, 20,  1, ...,  1, 16, 10],
       ...,
       [ 6,  1,  1, ...,  1, 12,  1],
       [12,  1,  1, ...,  4,  1,  1],
       [ 1,  4, 10, ..., 15,  2,  1]])

In [12]:
editable_array.max().max()

671160